In [ ]:
class excel:
    

    def kospi_stock_price(self):
    
        data = pd.read_excel('d:\\kospi_list.xlsx')

        code_list = data['종목코드'].tolist()
        code_list = [str(item).zfill(6) for item in code_list]
        name_list = data['종목명'].tolist()

        # 코스피 상장종목 전체
        stock_dic = dict(list(zip(code_list,name_list)))

        for code in stock_dic.keys():
            df  = fdr.DataReader(code,'2019-01-01')
            print(code,stock_dic[code])
            df['Code'],df['Name'] = code,stock_dic[code]
            df = df[['Code','Name','Open','High','Low','Volume','Close']]
    
            #df.to_excel('d:\\data_set\\kospi\\'+ stock_dic[code] +'.xlsx',engine = 'xlsxwriter')
            print(df)

In [17]:
from urllib.request import urlopen  
import datetime as dt
import bs4
index_cd = 'KPI200'

def date_format(d):
    #d = str(d).replace('-', '.')
    
    yyyy = int(d.split('-')[0]) 
    mm = int(d.split('-')[1])
    dd = int(d.split('-')[2])

    this_date= dt.date(yyyy, mm, dd)
    return this_date

class kpi_200:

    from urllib.request import urlopen  
    import bs4
    import datetime as dt

    
    page_n = 1
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)




    def historical_index_naver(self, index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
        if start_date:   # start_date가 있으면
            start_date = date_format(start_date)   # date 포맷으로 변환
        else:    # 없으면
            start_date = dt.date.today()   # 오늘 날짜를 지정
        if end_date:   
            end_date = date_format(end_date)   
        else:   
            end_date = dt.date.today()  
        
        
        naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    
        source = urlopen(naver_index).read()   # 지정한 페이지에서 코드 읽기
        source = bs4.BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
    
        dates = source.find_all('td', class_='date')   # <td class="date">태그에서 날짜 수집   
        prices = source.find_all('td', class_='number_1')   # <td class="number_1">태그에서 지수 수집
    
        for n in range(len(dates)):
    
            if dates[n].text.split('.')[0].isdigit():
            
                # 날짜 처리
                this_date = dates[n].text
                this_date= date_format(this_date)
            
                if this_date <= end_date and this_date >= start_date:   
                # start_date와 end_date 사이에서 데이터 저장
                    # 종가 처리
                    this_close = prices[n*4].text   # prices 중 종가지수인 0,4,8,...번째 데이터 추출
                    this_close = this_close.replace(',', '')
                    this_close = float(this_close)

                    # 딕셔너리에 저장
                    historical_prices[this_date] = this_close
                
                elif this_date < start_date:   
                # start_date 이전이면 함수 종료
                    return historical_prices              
            
        # 페이지 네비게이션
        if last_page == 0:
            last_page = source.find('td', class_='pgRR').find('a')['href']
            # 마지막페이지 주소 추출
            last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
            last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
            last_page = int(last_page)   # 숫자형 변수로 변환
        
        # 다음 페이지 호출
        if page_n < last_page:   
            page_n = page_n + 1   
            historical_index_naver(index_cd, start_date, end_date, page_n, last_page)   
        
        return historical_prices 




In [50]:
import pandas as pd
from pandas import DataFrame, Series
import requests as re
from bs4 import BeautifulSoup
import datetime as date
from urllib.request import urlopen

url = 'http://finance.naver.com/sise/sise_deposit.nhn?&page='    

source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

last = source.find('td',class_='pgRR').find('a')['href']
last = last.split('&')[1]
last = last.split('=')[1]
last = int(last)

# 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
folder_adress = 'd:\\'
file_name = 'pre_mondy_trend.xlsx'

def get_money_trend():

    #url = 'http://finance.naver.com/sise/sise_deposit.nhn?&page='    
    
    Data = DataFrame(columns = ['고객예탁금', '신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드'])

    # 날짜를 받을 리스트
    date_list = []

    
    # 값을 받을 사전
    dictionary = {'고객예탁금': [],
                  '신용잔고': [],
                  '주식형 펀드': [],
                  '혼합형 펀드': [],
                  '채권형 펀드': []
                  }

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['고객예탁금','신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드']


    # count mask
    mask = [0,1,3,5,7,9]
    
    for i in range(1,last+5):
        
        source = urlopen(url+str(i)).read()   # 지정한 페이지에서 코드 읽기
        source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
        
        tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})

        trs = tbody.find_all('tr')

        for tr in trs:
    
            tds = tr.find_all('td')
    
            count = 0
    
            for td in tds:
                
                # 변화량 제외하고 잔고량만 가져오기
                if not td.text.strip() == 0:
                    if len(td.text.strip()) >= 4:
                        
                        if count == 0:
                            date_ = td.text.strip().replace('.','-')
                            print('date_ = {}'.format(date_))
                            date_list.append(date_)
                            #print('date_list={}'.format(date_list))
                                                  
                        elif count in mask:
                            temp = int((count-1)/2)
                            print('temp={}'.format(temp))
                            dictionary[name_list[temp]].append(td.text.strip())
        
                    count += 1
                    print('count = {}'.format(count))
                
            if len(dictionary['고객예탁금']) != len(dictionary['주식형 펀드']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                dictionary['고객예탁금'].pop(-1)
                dictionary['신용잔고'].pop(-1)   
            
        if i > last:
            print(str(i-1) + '번째 페이지에서 크롤링 종료')
                                
            data = DataFrame(dictionary,index = date_list)
            data.to_excel(folder_adress + file_name, encoding = 'utf-8')
        
            return data                          
                    
        print(str(i) + '번째 페이지 크롤링 완료')            
                
get_money_trend()

count = 1
date_ = 19-01-23
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 19-01-22
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 19-01-21
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 19-01-18
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 19-01-17
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
count = 1
count = 1
count = 1
date_ = 19-01-16
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=

count = 1
date_ = 18-10-26
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-10-25
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-10-24
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-10-23
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-10-22
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
count = 1
count = 1
count = 1
date_ = 18-10-19
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=

count = 1
date_ = 18-07-26
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-07-25
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-07-24
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-07-23
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-07-20
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
count = 1
count = 1
count = 1
date_ = 18-07-19
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=

count = 1
date_ = 18-04-26
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-04-25
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-04-24
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-04-23
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-04-20
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
count = 1
count = 1
count = 1
date_ = 18-04-19
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=

count = 1
date_ = 18-01-29
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-01-26
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-01-25
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-01-24
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
date_ = 18-01-23
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=4
count = 10
count = 11
count = 1
count = 1
count = 1
date_ = 18-01-22
count = 1
temp=0
count = 2
count = 3
temp=1
count = 4
count = 5
temp=2
count = 6
count = 7
temp=3
count = 8
count = 9
temp=

KeyboardInterrupt: 

In [46]:
import pandas as pd
from pandas import DataFrame, Series
import requests as re
from bs4 import BeautifulSoup
import datetime as date

# 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
folder_adress = 'd:\\'

def get_money_trend():

    url = 'http://finance.naver.com/sise/sise_deposit.nhn?&page='    
    
    Data = DataFrame(columns = ['고객예탁금', '신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드'])

    # 날짜를 받을 리스트
    date_list = []

    
    # 값을 받을 사전
    dictionary = {'고객예탁금': [],
                  '신용잔고': [],
                  '주식형 펀드': [],
                  '혼합형 펀드': [],
                  '채권형 펀드': []
                  }

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['고객예탁금','신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드']


    # count mask
    mask = [0,1,3,5,7,9]
    
    for i in range(1,300):
        
        url_ = re.get(url+ str(i))
        url_ = url_.content
        html = BeautifulSoup(url_,'html.parser')

        tbody = html.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})

        trs = tbody.find_all('tr')

        for tr in trs:
    
            tds = tr.find_all('td')
    
            count = 0
    
            for td in tds:
                
                # 변화량 제외하고 잔고량만 가져오기
                if not td.text.strip() == 0:
                    if len(td.text.strip()) >= 4:
            
                        if count == 0:
                            date_ = td.text.strip().replace('.','-')
                        
                            if not date_ in date_list:
                                date_list.append(date_)
                        
                        
                        
                            #마지막 페이지인 경우
                            else:
                                print(str(i-1) + '번째 페이지에서 크롤링 종료')
                                
                                data = DataFrame(dictionary,index = date_list)
                                data.to_csv(folder_adress + '/money_trend.csv')
    
                                return data

                        
                        elif count in mask:
                            temp = int((count-1)/2)
                            dictionary[name_list[temp]].append(td.text.strip())
        
                    count += 1
            
            # 누락된 값을 발견하면 (펀드 자료에 누락된 값이 존재함)
            if len(dictionary['고객예탁금']) != len(dictionary['주식형 펀드']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                dictionary['고객예탁금'].pop(-1)
                dictionary['신용잔고'].pop(-1)
                
        print(str(i) + '번째 페이지 크롤링 완료')
        print(date_list)
get_money_trend()


1번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-12-24']
2번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-12-24', '18-12-21', '18-12-20', '18-12-19', '18-12-18', '18-12-17', '18-12-14', '18-12-13', '18-12-12', '18-12-11', '18-12-10', '18-12-07', '18-12-06', '18-12-05', '18-12-04', '18-12-03', '18-11-30', '18-11-29', '18-11-28', '18-11-27', '18-11-26']
3번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-

9번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-12-24', '18-12-21', '18-12-20', '18-12-19', '18-12-18', '18-12-17', '18-12-14', '18-12-13', '18-12-12', '18-12-11', '18-12-10', '18-12-07', '18-12-06', '18-12-05', '18-12-04', '18-12-03', '18-11-30', '18-11-29', '18-11-28', '18-11-27', '18-11-26', '18-11-23', '18-11-22', '18-11-21', '18-11-20', '18-11-19', '18-11-16', '18-11-15', '18-11-14', '18-11-13', '18-11-12', '18-11-09', '18-11-08', '18-11-07', '18-11-06', '18-11-05', '18-11-02', '18-11-01', '18-10-31', '18-10-30', '18-10-29', '18-10-26', '18-10-25', '18-10-24', '18-10-23', '18-10-22', '18-10-19', '18-10-18', '18-10-17', '18-10-16', '18-10-15', '18-10-12', '18-10-11', '18-10-10', '18-10-08', '18-10-05', '18-10-04', '18-10-02', '18-10-01', '18-09-28', '18-09-27', '18-09-21', '18-09-20', 

13번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-12-24', '18-12-21', '18-12-20', '18-12-19', '18-12-18', '18-12-17', '18-12-14', '18-12-13', '18-12-12', '18-12-11', '18-12-10', '18-12-07', '18-12-06', '18-12-05', '18-12-04', '18-12-03', '18-11-30', '18-11-29', '18-11-28', '18-11-27', '18-11-26', '18-11-23', '18-11-22', '18-11-21', '18-11-20', '18-11-19', '18-11-16', '18-11-15', '18-11-14', '18-11-13', '18-11-12', '18-11-09', '18-11-08', '18-11-07', '18-11-06', '18-11-05', '18-11-02', '18-11-01', '18-10-31', '18-10-30', '18-10-29', '18-10-26', '18-10-25', '18-10-24', '18-10-23', '18-10-22', '18-10-19', '18-10-18', '18-10-17', '18-10-16', '18-10-15', '18-10-12', '18-10-11', '18-10-10', '18-10-08', '18-10-05', '18-10-04', '18-10-02', '18-10-01', '18-09-28', '18-09-27', '18-09-21', '18-09-20',

16번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-12-24', '18-12-21', '18-12-20', '18-12-19', '18-12-18', '18-12-17', '18-12-14', '18-12-13', '18-12-12', '18-12-11', '18-12-10', '18-12-07', '18-12-06', '18-12-05', '18-12-04', '18-12-03', '18-11-30', '18-11-29', '18-11-28', '18-11-27', '18-11-26', '18-11-23', '18-11-22', '18-11-21', '18-11-20', '18-11-19', '18-11-16', '18-11-15', '18-11-14', '18-11-13', '18-11-12', '18-11-09', '18-11-08', '18-11-07', '18-11-06', '18-11-05', '18-11-02', '18-11-01', '18-10-31', '18-10-30', '18-10-29', '18-10-26', '18-10-25', '18-10-24', '18-10-23', '18-10-22', '18-10-19', '18-10-18', '18-10-17', '18-10-16', '18-10-15', '18-10-12', '18-10-11', '18-10-10', '18-10-08', '18-10-05', '18-10-04', '18-10-02', '18-10-01', '18-09-28', '18-09-27', '18-09-21', '18-09-20',

19번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-12-24', '18-12-21', '18-12-20', '18-12-19', '18-12-18', '18-12-17', '18-12-14', '18-12-13', '18-12-12', '18-12-11', '18-12-10', '18-12-07', '18-12-06', '18-12-05', '18-12-04', '18-12-03', '18-11-30', '18-11-29', '18-11-28', '18-11-27', '18-11-26', '18-11-23', '18-11-22', '18-11-21', '18-11-20', '18-11-19', '18-11-16', '18-11-15', '18-11-14', '18-11-13', '18-11-12', '18-11-09', '18-11-08', '18-11-07', '18-11-06', '18-11-05', '18-11-02', '18-11-01', '18-10-31', '18-10-30', '18-10-29', '18-10-26', '18-10-25', '18-10-24', '18-10-23', '18-10-22', '18-10-19', '18-10-18', '18-10-17', '18-10-16', '18-10-15', '18-10-12', '18-10-11', '18-10-10', '18-10-08', '18-10-05', '18-10-04', '18-10-02', '18-10-01', '18-09-28', '18-09-27', '18-09-21', '18-09-20',

21번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-12-24', '18-12-21', '18-12-20', '18-12-19', '18-12-18', '18-12-17', '18-12-14', '18-12-13', '18-12-12', '18-12-11', '18-12-10', '18-12-07', '18-12-06', '18-12-05', '18-12-04', '18-12-03', '18-11-30', '18-11-29', '18-11-28', '18-11-27', '18-11-26', '18-11-23', '18-11-22', '18-11-21', '18-11-20', '18-11-19', '18-11-16', '18-11-15', '18-11-14', '18-11-13', '18-11-12', '18-11-09', '18-11-08', '18-11-07', '18-11-06', '18-11-05', '18-11-02', '18-11-01', '18-10-31', '18-10-30', '18-10-29', '18-10-26', '18-10-25', '18-10-24', '18-10-23', '18-10-22', '18-10-19', '18-10-18', '18-10-17', '18-10-16', '18-10-15', '18-10-12', '18-10-11', '18-10-10', '18-10-08', '18-10-05', '18-10-04', '18-10-02', '18-10-01', '18-09-28', '18-09-27', '18-09-21', '18-09-20',

23번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-12-24', '18-12-21', '18-12-20', '18-12-19', '18-12-18', '18-12-17', '18-12-14', '18-12-13', '18-12-12', '18-12-11', '18-12-10', '18-12-07', '18-12-06', '18-12-05', '18-12-04', '18-12-03', '18-11-30', '18-11-29', '18-11-28', '18-11-27', '18-11-26', '18-11-23', '18-11-22', '18-11-21', '18-11-20', '18-11-19', '18-11-16', '18-11-15', '18-11-14', '18-11-13', '18-11-12', '18-11-09', '18-11-08', '18-11-07', '18-11-06', '18-11-05', '18-11-02', '18-11-01', '18-10-31', '18-10-30', '18-10-29', '18-10-26', '18-10-25', '18-10-24', '18-10-23', '18-10-22', '18-10-19', '18-10-18', '18-10-17', '18-10-16', '18-10-15', '18-10-12', '18-10-11', '18-10-10', '18-10-08', '18-10-05', '18-10-04', '18-10-02', '18-10-01', '18-09-28', '18-09-27', '18-09-21', '18-09-20',

25번째 페이지 크롤링 완료
['19-01-23', '19-01-22', '19-01-21', '19-01-18', '19-01-17', '19-01-16', '19-01-15', '19-01-14', '19-01-11', '19-01-10', '19-01-09', '19-01-08', '19-01-07', '19-01-04', '19-01-03', '19-01-02', '18-12-28', '18-12-27', '18-12-26', '18-12-24', '18-12-21', '18-12-20', '18-12-19', '18-12-18', '18-12-17', '18-12-14', '18-12-13', '18-12-12', '18-12-11', '18-12-10', '18-12-07', '18-12-06', '18-12-05', '18-12-04', '18-12-03', '18-11-30', '18-11-29', '18-11-28', '18-11-27', '18-11-26', '18-11-23', '18-11-22', '18-11-21', '18-11-20', '18-11-19', '18-11-16', '18-11-15', '18-11-14', '18-11-13', '18-11-12', '18-11-09', '18-11-08', '18-11-07', '18-11-06', '18-11-05', '18-11-02', '18-11-01', '18-10-31', '18-10-30', '18-10-29', '18-10-26', '18-10-25', '18-10-24', '18-10-23', '18-10-22', '18-10-19', '18-10-18', '18-10-17', '18-10-16', '18-10-15', '18-10-12', '18-10-11', '18-10-10', '18-10-08', '18-10-05', '18-10-04', '18-10-02', '18-10-01', '18-09-28', '18-09-27', '18-09-21', '18-09-20',

KeyboardInterrupt: 

In [40]:
import pandas as pd
from pandas import DataFrame, Series
import requests as re
from bs4 import BeautifulSoup
import datetime as date
from urllib.request import urlopen

url = 'http://finance.naver.com/sise/sise_deposit.nhn?&page=1'    

source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류

last = source.find('td',class_='pgRR').find('a')['href']
last = last.split('&')[1]
last = last.split('=')[1]
last = int(last)

# 사용자의 PC내 폴더 주소를 입력하시면 됩니다.
folder_adress = 'd:\\'
file_name = 'pre_mondy_trend.xlsx'

def get_money_trend():

    url = 'http://finance.naver.com/sise/sise_deposit.nhn?&page='    
    
    Data = DataFrame(columns = ['고객예탁금', '신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드'])

    # 날짜를 받을 리스트
    date_list = []

    
    # 값을 받을 사전
    dictionary = {'고객예탁금': [],
                  '신용잔고': [],
                  '주식형 펀드': [],
                  '혼합형 펀드': [],
                  '채권형 펀드': []
                  }

    # dictionary key 인덱싱을 위한 리스트
    name_list = ['고객예탁금','신용잔고','주식형 펀드','혼합형 펀드','채권형 펀드']


    # count mask
    mask = [0,1,3,5,7,9]
    
    for i in range(1,last+5):
        
        source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
        source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
        
        tbody = source.find('div',{'id':'wrap'}).find('div',{'class':'box_type_m'})

        trs = tbody.find_all('tr')

        for tr in trs:
    
            tds = tr.find_all('td')
    
            count = 0
    
            for td in tds:
                
                # 변화량 제외하고 잔고량만 가져오기
                if not td.text.strip() == 0:
                    if len(td.text.strip()) >= 4:
            
                        if count == 0:
                            date_ = td.text.strip().replace('.','-')
                        
                            #print("loop : {}".format(i))
                            date_list.append(date_)
                        
                      
                        elif count in mask:
                            temp = int((count-1)/2)
                            dictionary[name_list[temp]].append(td.text.strip())
        
                    count += 1
            
            # 누락된 값을 발견하면 (펀드 자료에 누락된 값이 존재함)
            if len(dictionary['고객예탁금']) != len(dictionary['주식형 펀드']):
                print(str(i)+ '번째 페이지에서 누락된 값 발생')
                print('누락된 데이터를 제거합니다')
                    
                date_list.pop(-1)
                dictionary['고객예탁금'].pop(-1)
                dictionary['신용잔고'].pop(-1)
                
        if i > last:
            print(str(i-1) + '번째 페이지에서 크롤링 종료')
                                
            data = DataFrame(dictionary,index = date_list)
            data.to_excel(folder_adress + file_name, encoding = 'utf-8')
        
            return data        
    
data = get_money_trend()

df = pd.read_excel('d:\\pre_mondy_trend.xlsx')
df = df.sort_index()
df.to_excel('d:\\money_trends.xlsx')
       


KeyboardInterrupt: 

In [19]:
historical_prices = dict()

In [20]:
a.historical_index_naver(index_cd, '2019-1-1')

ValueError: invalid literal for int() with base 10: '2019.01.25'

In [ ]:
index_cd = 'KPI200'
historical_prices = dict()
historical_index_naver(index_cd, '2019-1-1')
historical_prices

In [4]:
index_cd = 'KPI200'
page_n = 1
naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)

from urllib.request import urlopen  
source = urlopen(naver_index).read()
import datetime as dt
import bs4
source = bs4.BeautifulSoup(source, 'lxml')

def date_format(d):
    d = str(d).replace('-', '.')
    
    yyyy = int(d.split('-')[0]) 
    mm = int(d.split('-')[1])
    dd = int(d.split('-')[2])

    this_date= dt.date(yyyy, mm, dd)
    return this_date

def historical_index_naver(index_cd, start_date='', end_date= '', page_n=1, last_page=0):
    
    if start_date:   # start_date가 있으면
        start_date = date_format(start_date)   # date 포맷으로 변환
    else:    # 없으면
        start_date = dt.date.today()   # 오늘 날짜를 지정
    if end_date:   
        end_date = date_format(end_date)   
    else:   
        end_date = dt.date.today()  
        
        
    naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_index).read()   # 지정한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
    
    dates = source.find_all('td', class_='date')   # <td class="date">태그에서 날짜 수집   
    prices = source.find_all('td', class_='number_1')   # <td class="number_1">태그에서 지수 수집
    
    for n in range(len(dates)):
    
        if dates[n].text.split('.')[0].isdigit():
            
            # 날짜 처리
            this_date = dates[n].text
            this_date= date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:   
            # start_date와 end_date 사이에서 데이터 저장
                # 종가 처리
                this_close = prices[n*4].text   # prices 중 종가지수인 0,4,8,...번째 데이터 추출
                this_close = this_close.replace(',', '')
                this_close = float(this_close)

                # 딕셔너리에 저장
                historical_prices[this_date] = this_close
                
            elif this_date < start_date:   
            # start_date 이전이면 함수 종료
                return historical_prices              
            
    # 페이지 네비게이션
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        # 마지막페이지 주소 추출
        last_page = last_page.split('&')[1]   # & 뒤의 page=506 부분 추출
        last_page = last_page.split('=')[1]   # = 뒤의 페이지번호만 추출
        last_page = int(last_page)   # 숫자형 변수로 변환
        
    # 다음 페이지 호출
    if page_n < last_page:   
        page_n = page_n + 1   
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)   
        
    return historical_prices  


index_cd = 'KPI200'
historical_prices = dict()
historical_index_naver(index_cd, '2019-1-1')
historical_prices

ValueError: invalid literal for int() with base 10: '2019.1.1'

In [13]:
def date_format(d):
    #d = str(d).replace('-', '.')
    
    yyyy = int(d.split('-')[0]) 
    mm = int(d.split('-')[1])
    dd = int(d.split('-')[2])

    this_date= dt.date(yyyy, mm, dd)
    return this_date

In [14]:
now = date_format('2018-01-02')
now


datetime.date(2018, 1, 2)

In [1]:
import mod1

SyntaxError: invalid syntax (mod1.py, line 130)

In [2]:
a = mod1.excel()

In [3]:
a.kpi_200('KPI200', '2019-1-1','2019-1-10')

NameError: name 'kpi_200' is not defined